In [1]:
!pip install keras-tuner


     |████████████████████████████████| 71kB 8.0MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=bba91c9e2b9a02e85738611e8ee512264fc7df9c44b882fe7574c088559ee6c9
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=2e40038cc1543074eae97e468fcab6351833803938b24b7f7d39eae328b90d46
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [2]:

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
train_images=train_images/255
test_images=test_images/255

In [6]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [12]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [13]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [14]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [19]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [20]:
model=tuner_search.get_best_models(num_models=1)[0]

In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 24, 24, 96)        2496      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 48)        115248    
_________________________________________________________________
flatten (Flatten)            (None, 19200)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               2150512   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 2,269,386
Trainable params: 2,269,386
Non-trainable params: 0
_________________________________________________________________


In [22]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 8s 5ms/step - loss: 0.1817 - accuracy: 0.9339 - val_loss: 0.2555 - val_accuracy: 0.9063
Epoch 5/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.1368 - accuracy: 0.9494 - val_loss: 0.2800 - val_accuracy: 0.9072
Epoch 6/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0983 - accuracy: 0.9627 - val_loss: 0.3296 - val_accuracy: 0.9035
Epoch 7/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0768 - accuracy: 0.9711 - val_loss: 0.3516 - val_accuracy: 0.9067
Epoch 8/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0568 - accuracy: 0.9782 - val_loss: 0.4021 - val_accuracy: 0.9060
Epoch 9/10
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0472 - accuracy: 0.9832 - val_loss: 0.4436 - val_accuracy: 0.9038
Epoch 10/10
1688/1688 [==============================] - 8s 4ms/step - loss: 0.0339 - accuracy: 0.9889 - val_loss: 0.4901 - val_accuracy